# Artificial Neural Network (ANN)

Deep Neural Network (DNN) Regressor

3 layers: 1) Input 2) Hidden and 3) Output

feature: Input data to the network(features)

label: output from the network (labels)

loss function: Metric used to estimate the performance of the learning phase

optimizer: Improve the learning by updating the knowledge in the network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

## (1) Import the data

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
# Create more data
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Return
Date,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,-1,0.012107
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,-1,-1,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,-1,1,-0.021531


In [4]:
dataset.shape

(1171, 10)

In [5]:
X = dataset.drop(['Adj Close', 'Close'], axis=1)  
y = dataset['Adj Close'] 

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train  = y_train.astype(int)
y_test  = y_test.astype(int)
batch_size =len(X_train)

print(X_train.shape, y_train.shape,y_test.shape)

(936, 8) (936,) (235,)


## (2) Transform the data

In [7]:
## rescale
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Train
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# test
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

In [28]:
print(X_train_scaled.shape, X_test_scaled.shape)

(936, 8) (235, 8)


## (3) Construct the tensor

In [8]:
import tensorflow as tf

feature_columns = [tf.feature_column.numeric_column('x', shape=X_train_scaled.shape[1:])]		

## (4) Build the model

In [32]:
estimator = tf.estimator.DNNRegressor(
    feature_columns=feature_columns,
    hidden_units=[300, 100])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': 600, '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029778509710>, '_task_type': 'worker', '_device_fn': None, '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\TINHAN~1\\AppData\\Local\\Temp\\tmp40fu3k68', '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_train_distribute': None}


## (5) Train and evaluate the model

In [33]:
# Train the estimator
train_input = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_scaled},
    y=y_train,
    batch_size=50,
    shuffle=False,
    num_epochs=None)
estimator.train(input_fn = train_input,steps=1000) 
eval_input = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_scaled},
    y=y_test, 
    shuffle=False,
    batch_size=X_test_scaled.shape[0],
    num_epochs=1)
estimator.evaluate(eval_input,steps=None) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\TINHAN~1\AppData\Local\Temp\tmp40fu3k68\model.ckpt.
INFO:tensorflow:loss = 3834.94, step = 1
INFO:tensorflow:global_step/sec: 680.489
INFO:tensorflow:loss = 6.5443, step = 101 (0.149 sec)
INFO:tensorflow:global_step/sec: 869.846
INFO:tensorflow:loss = 5.31314, step = 201 (0.115 sec)
INFO:tensorflow:global_step/sec: 806.707
INFO:tensorflow:loss = 6.1748, step = 301 (0.124 sec)
INFO:tensorflow:global_step/sec: 840.608
INFO:tensorflow:loss = 4.62847, step = 401 (0.119 sec)
INFO:tensorflow:global_step/sec: 800.255
INFO:tensorflow:loss = 5.12176, step = 501 (0.134 sec)
INFO:tensorflow:global_step/sec: 813.267
INFO:tensorflow:loss = 5.35003, step = 601 (0.114 sec)
INFO:tensorflow:global_step/sec: 775.443


{'average_loss': 6.4982734, 'global_step': 1000, 'loss': 1527.0942}

## (6) Improve the model

In [36]:
estimator_imp = tf.estimator.DNNRegressor(
    feature_columns=feature_columns,
    hidden_units=[300, 100],
    dropout=0.3,
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.01, 
      l2_regularization_strength=0.01
    ))
estimator_imp.train(input_fn = train_input,steps=1000) 
estimator_imp.evaluate(eval_input,steps=None) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': 600, '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_evaluation_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029778EC56A0>, '_task_type': 'worker', '_device_fn': None, '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\TINHAN~1\\AppData\\Local\\Temp\\tmpoan4nk2h', '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_train_distribute': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 

{'average_loss': 6.139895, 'global_step': 1000, 'loss': 1442.8754}